In [135]:
!pip install pydicom opencv-python
import os
import pydicom
import numpy as np
import scipy.ndimage as ndi
import glob
import matplotlib.pyplot as plt

import cv2


# Function to load DICOM files from a folder
def load_dicom_files(folder_path):
    dicom_files = []
    for file_path in glob.glob(os.path.join(folder_path, '*.dcm')):
        dicom_files.append(pydicom.dcmread(file_path))
    return dicom_files



def lung_segmentation(image, threshold_value):
    # Apply thresholding
    _, binary_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)

    # Perform morphological operations (e.g., closing) to fill holes
    kernel = np.ones((5, 5), np.uint8)
    closed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)

    return closed_image


def calculate_lung_volume(segmented_image, pixel_spacing_x, pixel_spacing_y, slice_thickness):
    # Calculate pixel count
    pixel_count = np.count_nonzero(segmented_image)

    # Calculate volume (in cubic millimeters)
    volume_mm3 = pixel_count * pixel_spacing_x * pixel_spacing_y * slice_thickness

    # Convert volume to liters
    volume_liters = volume_mm3 / 1000.0

    return volume_liters


# Define the patient folders
patient_folders = [
    r'/content/drive/MyDrive/의료영상분석_실습자료/중간프로젝트/1',
    r'/content/drive/MyDrive/의료영상분석_실습자료/중간프로젝트/2',
    r'/content/drive/MyDrive/의료영상분석_실습자료/중간프로젝트/3',
    r'/content/drive/MyDrive/의료영상분석_실습자료/중간프로젝트/4',
    r'/content/drive/MyDrive/의료영상분석_실습자료/중간프로젝트/5',
]

In [140]:
inspiration_thickness = 1.0  # mm
expiration_thickness = 3.0   # mm

# Iterate through patient folders
patient_folder = patient_folders[２｣
# Load DICOM files
dicom_files = load_dicom_files(patient_folder)

SyntaxError: ignored

In [138]:
# Separate DICOM files for inspiration (1mm) and expiration (3mm)
inspiration_dicom = []
expiration_dicom = []
for dcm in dicom_files:

    if dcm.SeriesDescription == 'HR Insp Spiral  1.0  B45f':
        inspiration_dicom.append(dcm)

    if dcm.SeriesDescription == 'HR Exp Spiral  1.0  B45f':
        expiration_dicom.append(dcm)


# Define threshold values for lung segmentation (adjust as needed)
inspiration_threshold = 400  # Adjust as needed
expiration_threshold = 400   # Adjust as needed

# Apply lung segmentation to inspiration and expiration images
segmented_inspiration = lung_segmentation(inspiration_dicom[0].pixel_array, inspiration_threshold)
segmented_expiration = lung_segmentation(expiration_dicom[0].pixel_array, expiration_threshold)



# Define pixel spacing (replace with actual values)
pixel_spacing_x = float(inspiration_dicom[0]['0028', '0030'][0])
pixel_spacing_y = float(inspiration_dicom[0]['0028', '0030'][1])
slice_thickness = float(inspiration_dicom[0]['0018', '0050'].value) if inspiration_dicom else None

# Calculate volumes for inspiration and expiration lung segmentations
inspiration_lung_volume = calculate_lung_volume(segmented_inspiration, pixel_spacing_x, pixel_spacing_y, slice_thickness)
expiration_lung_volume = calculate_lung_volume(segmented_expiration, pixel_spacing_x, pixel_spacing_y, slice_thickness)


# Calculate volume difference (FVC) in cc
fvc = inspiration_lung_volume - expiration_lung_volume

# Print or store the results for this patient
print(f"Patient in folder {patient_folder}:")
print(f"Inspiration Lung Volume: {inspiration_lung_volume} cc")
print(f"Expiration Lung Volume: {expiration_lung_volume} cc")
print(f"Volume Difference (FVC): {fvc} cc")
print("-" * 40)

Patient in folder /content/drive/MyDrive/의료영상분석_실습자료/중간프로젝트/2:
Inspiration Lung Volume: 40.06724332427979 cc
Expiration Lung Volume: 40.98768316268921 cc
Volume Difference (FVC): -0.9204398384094219 cc
----------------------------------------


In [ ]:
Patient in folder /content/drive/MyDrive/의료영상분석_실습자료/중간프로젝트/2:
Inspiration Lung Volume: 40.06724332427979 cc
Expiration Lung Volume: 40.98768316268921 cc
Volume Difference (FVC): -0.9204398384094219 cc